In [1]:
import pandas as pd
import numpy as np
import os
import random
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import Dataset, DataLoader

In [10]:
class TUAPDataset(Dataset):
    """Dataset for TUAP."""

    def __init__(self, csv_path, sequence_length, train=True, normalize=True):
        """Initialize the dataset.

        Parameters
        ----------
        csv_path : str
            Path to the csv file.
        sequence_length : int
            Length of the sequence.
        train : bool, optional
            Whether to use the train set or the test set, by default True
        normalize : bool, optional
            Whether to normalize the data, by default True
        """
        self.data = pd.read_csv(csv_path)
        self.sequence_length = sequence_length
        # self.data['hour'] = self.data['timestamp'].apply(lambda x: int(x.split(' ')[1].split(':')[0]))
        self.data['date'] = self.data['timestamp'].apply(lambda x: x.split(' ')[0])

        # Get all the data for each day
        self.data = self.data.groupby('date').apply(lambda x: x.values).values

        # Drop datetime column
        self.data = np.array([x[:, 1:-1] for x in self.data])
        
        
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """Get the item at the given index."""
        return self.data

In [37]:
symbol='AAPL'
dataset = TUAPDataset(csv_path=f'dataset/splitted_s&p500/{symbol}.csv', sequence_length=5, train=True, normalize=False)

/var/folders/lk/512hb8bd7ld3l5z5s_8d4b7c0000gn/T/ipykernel_6194/1835614381.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.data = np.array([x[:, 1:-1] for x in self.data])


In [38]:
x = dataset[0]

In [39]:
x = np.concatenate([x[0], x[1], x[2]], axis=0)

In [55]:
max_idx = len(x)-35
idx_arr = np.random.uniform(0,max_idx,120)
samples = [x[int(i):int(i)+35] for i in idx_arr]

In [66]:
samples = np.array(samples)

In [76]:
samples[0,1]

array([159.03, 159.07, 159.01, 159.03, 32760.0], dtype=object)

In [77]:
samples[:, 1, 4]

array([32760.0, 20842.0, 165571.0, 40125.0, 110001.0, 32729.0, 20368.0,
       125560.0, 166104.0, 430885.0, 184642.0, 202289.0, 289582.0,
       20186.0, 83954.0, 150958.0, 123444.0, 70595.0, 155470.0, 26447.0,
       208900.0, 186812.0, 54054.0, 175115.0, 24401.0, 197132.0, 64190.0,
       256854.0, 138548.0, 57484.0, 138816.0, 126531.0, 18674.0, 49520.0,
       104130.0, 89110.0, 39486.0, 25742.0, 62680.0, 41340.0, 117504.0,
       343254.0, 49738.0, 199332.0, 93332.0, 24499.0, 39671.0, 78808.0,
       59204.0, 333048.0, 139917.0, 202289.0, 94952.0, 53250.0, 27610.0,
       42508.0, 105799.0, 179682.0, 37992.0, 184182.0, 42063.0, 117504.0,
       63214.0, 38907.0, 66009.0, 85595.0, 52541.0, 219161.0, 35969.0,
       32708.0, 135657.0, 405334.0, 285947.0, 77575.0, 48084.0, 190257.0,
       135304.0, 65813.0, 45180.0, 149598.0, 255539.0, 41413.0, 390365.0,
       352803.0, 135383.0, 59479.0, 72936.0, 48943.0, 32973.0, 45059.0,
       42474.0, 85166.0, 99391.0, 236697.0, 68975.0, 71444

In [85]:
input_shape = (35, 7)

In [86]:
mask = np.zeros(input_shape,dtype=np.float32)
for i in range(25):
    mask[i+5,3] = 1.

In [88]:
data = np.zeros((8,35,7), dtype=np.float32)
target = np.zeros((8,2), dtype=np.float32)
global_uap = np.zeros(input_shape, dtype=np.float32)
perturbation = np.zeros(input_shape, dtype=np.float32)

x_adv = data + (global_uap + perturbation)*mask 

In [90]:
x_adv

(8, 35, 7)

In [ ]:
# Get the first batch of data
x, y = next(iter(train_loader))

# Get the loss function
criterion = nn.BCELoss()

fooling_targets = [torch.tensor([0., 1.]), torch.tensor([1., 0.])]

current_target = fooling_targets[0]

# Create perturbation to the xi
delta = torch.zeros_like(x[0], requires_grad=True)

for xi in x:
    # Add the perturbation to the xi
    x_perturbed = xi + delta

    # Format data for model
    input_perturbed = create_input_data(x_perturbed.detach().numpy())
    input_xp = input_perturbed[:, :-2]
    input_xp = torch.from_numpy(input_xp).float()

    # Get the model's prediction
    y_pred_perturbed = model(input_xp)
    y_pred_perturbed = torch.sigmoid(y_pred_perturbed)

    # Duplicate the current_target to match the shape of y_pred_perturbed
    target = current_target.repeat(y_pred_perturbed.shape[0], 1)

    # Calculate the loss
    loss = criterion(y_pred_perturbed, target)

    # Calculate the gradients for delta
    loss.backward()

    # Update delta
    delta.data = delta.data + 0.01 * delta.grad.data

    # Zero the gradients
    delta.grad.data.zero_()
    
    # Check if the model's prediction is the same as the target
    if torch.all(torch.eq(y_pred_perturbed, target)):
        print("Found the perturbation!")
        break

print(f"delta: {delta}")


pass
